In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df=pd.read_csv("PCOS ANALYSIS FINAL.csv")


df.columns=['time','diagnosed','result','age','overweight','weightgain','periods','conceiving','chinHair','cheeksHair','upperLipHair','betweenBreastHair','armsHair','innerThighHair','acneOrskinTag','hairThinning','darkPatch','tiredness','moodSwings','exercise','eatOutside','cannedFood','city']
df.dtypes


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
new=pd.DataFrame(df[df.diagnosed=='Yes'])
new.reset_index(drop=True,inplace=True)
new
new.drop('time',axis=1,inplace=True)
new
new.weightgain.fillna('abc',inplace=True)
y=pd.DataFrame(new.result)
new.drop('result',axis=1,inplace=True)
new
def g(s):
    if(s=="YES"):
        return 1
    if(s=="NO"):
        return 0
    if s=="Yes":
        return 1
    if s=="No":
        return 0
    if s=="abc":
        return 2
    return 0

new["Diagnose"]=new.diagnosed.apply(g)
new["Overweight"]=new.overweight.apply(g)
new["Weightgain"]=new.weightgain.apply(g)
new["Periods"]=new.periods.apply(g)
new["Conceiving"]=new.conceiving.apply(g)
new["AcneOrskinTag"]=new.acneOrskinTag.apply(g)
new["HairThinning"]=new.hairThinning.apply(g)
new["DarkPatch"]=new.darkPatch.apply(g)
new["Tiredness"]=new.tiredness.apply(g)
new["MoodSwings"]=new.moodSwings.apply(g)
new["CannedFood"]=new.cannedFood.apply(g)
new["City"]=new.city.apply(g)
del new["diagnosed"]
del new["overweight"]
del new["weightgain"]
del new["periods"]
del new["conceiving"]
del new["acneOrskinTag"]
del new["hairThinning"]
del new["darkPatch"]
del new["tiredness"]
del new["moodSwings"]
del new["cannedFood"]
del new["city"]
new.head()

def f(s):
    if(s=="Yes"):
        return True
    if(s=="No"):
        return False
    if(s=="Yes(Detected Positive)"):
        return True
    if(s=="No(Detected Negative)"):
        return False
y=y.result.apply(f)

y.head()
#converting dataframe to array
xnew=new.values
ynew=y.values

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(copy=True)
scaler.fit(xnew)
x_scaled=scaler.transform(xnew)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(xnew,ynew,random_state=3)
############SVC ALGORITHM########################################
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC
clf=SVC(kernel='poly',degree=2,C=0.01)
clf.fit(x_train,y_train)
ypredsvm=clf.predict(x_test)
clf.score(x_train,y_train),clf.score(x_test,y_test)
############Decision Tree ALGORITHM########################################
from sklearn.tree import DecisionTreeClassifier
clf1=DecisionTreeClassifier()
clf1.fit(x_train,y_train)
ypredDT=clf1.predict(x_test)
clf1.score(x_train,y_train),clf1.score(x_test,y_test)
new.columns
clf1.feature_importances_
############Naive Bayse ALGORITHM########################################
from sklearn.naive_bayes import GaussianNB
clf2=GaussianNB()
clf2.fit(x_train,y_train)
ypredNB=clf2.predict(x_test)
clf2.score(x_train,y_train),clf2.score(x_test,y_test)
############Linear Regression ALGORITHM########################################
from sklearn.linear_model import LinearRegression
clf5=LinearRegression()
clf5.fit(x_train,y_train)
ypredLR=clf5.predict(x_test)
clf5.score(x_train,y_train),clf5.score(x_test,y_test)
############Logistic Regression ALGORITHM########################################
from sklearn.linear_model import LogisticRegression
clf3 = LogisticRegression(C=0.01)
clf3.fit(x_train,y_train)
ypredLR=clf3.predict(x_test)
clf3.score(x_train,y_train),clf3.score(x_test,y_test)
############KNN ALGORITHM########################################
from sklearn.neighbors import KNeighborsClassifier
clf6=KNeighborsClassifier(n_neighbors=3)
clf6.fit(x_train,y_train)
ypredKNN=clf6.predict(x_test)
clf6.score(x_train,y_train),clf6.score(x_test,y_test)
############Random Forest ALGORITHM########################################
from sklearn.ensemble import RandomForestClassifier
clf7 = RandomForestClassifier(criterion="entropy")
clf7.fit(x_train, y_train)
ypredRF=clf7.predict(x_test)
clf7.score(x_train,y_train),clf7.score(x_test,y_test)
from sklearn.model_selection import StratifiedKFold
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((test.shape[0],1),float)
    train_pred=np.empty((0,1),float)
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]
        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        test_pred=np.append(test_pred,model.predict(test))
    return test_pred.reshape(-1,1),train_pred

,age,chinHair,cheeksHair,upperLipHair,betweenBreastHair,armsHair,innerThighHair,exercise,eatOutside,Diagnose,Overweight,Weightgain,Periods,Conceiving,AcneOrskinTag,HairThinning,DarkPatch,Tiredness,MoodSwings,CannedFood,City
0,23.0,3.0,4.0,4.0,3.0,2.0,3.0,3.0,4.0,1,0,2,0,0,0,0,0,1,1,1,0
1,22.0,4.0,1.0,3.0,2.0,2.0,1.0,4.0,3.0,1,1,0,0,0,1,1,0,1,1,0,0
2,26.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,1,0,2,0,0,0,1,0,0,1,0,0
3,20.0,3.0,2.0,4.0,3.0,3.0,3.0,5.0,2.0,1,1,1,1,0,0,0,1,1,1,0,1
4,19.0,3.0,1.0,4.0,3.0,3.0,3.0,2.0,2.0,1,0,2,1,0,1,1,0,1,1,1,0


In [ ]:
xtrain=pd.DataFrame(x_train)
ytrain=pd.DataFrame(y_train)
xtest=pd.DataFrame(x_test)
##########K Fold Validation ####################################
test_pred1 ,train_pred1=Stacking(model=clf6,n_fold=5, train=xtrain,test=xtest,y=ytrain)
test_pred2 ,train_pred2=Stacking(model=clf1,n_fold=5, train=xtrain,test=xtest,y=ytrain)
test_pred3 ,train_pred3=Stacking(model=clf,n_fold=5, train=xtrain,test=xtest,y=ytrain)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)

test_pred1=test_pred1.iloc[:41]
test_pred2=test_pred2.iloc[:41]
test_pred3=test_pred3.iloc[:41]

df_train = pd.concat([train_pred1, train_pred2, train_pred3], axis=1)
df_test = pd.concat([test_pred1, test_pred2,test_pred3], axis=1)

ytest=pd.DataFrame(y_test)
model = RandomForestClassifier(random_state=1,criterion="entropy")
model.fit(df_train,y_train)
model.score(df_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launche

0.7560975609756098

In [ ]:


#Bagging 
from sklearn. ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
bg = BaggingClassifier(clf3, max_samples= 0.5, max_features = 1.0, n_estimators = 20)
bg.fit(x_train,y_train)
bg.score(x_test,y_test)


0.7560975609756098

In [ ]:
#Boosting - Ada Boost
adb = AdaBoostClassifier(LogisticRegression(),n_estimators = 5, learning_rate = 1)
adb.fit(x_train,y_train)
adb.score(x_test,y_test)

0.7560975609756098

In [ ]:
evc = VotingClassifier( estimators= [('svm',clf),('dt',clf1),('nb',clf2),('lr',clf3),('knn',clf6)], voting = 'hard')
evc.fit(x_train,y_train)
evc.score(x_test, y_test)
x_train[0]
###working directory
from sklearn.externals import joblib  # Save to file
joblib_file = "model.pkl"   
joblib.dump(evc, joblib_file)
from keras.models import load_model
loaded_model = joblib.load('./model.pkl')
##Input ############
age = int(input('Select your age at which you had PCOS'))
Chin = int(input('Chin(Rate from 1-low to 5-high)')) 	
Cheeks = int(input('Cheeks(Rate from 1-low to 5-high)')) 	
Lips = int(input('Upper Lips(Rate from 1-low to 5-high)')) 	
Breast = int(input('Between breast(Rate from 1-low to 5-high)')) 	
Arms = int(input('Arms(Rate from 1-low to 5-high)')) 	
Thigh = int(input('Inner thighs(Rate from 1-low to 5-high)'))
BMI = (input('Were you overweight according to your BMI (Body Mass Index) ?'))
Weight = (input('Have you experienced significant or rapid weight gain and faced difficulty in losing it ?'))
Period = (input('Have you experienced irregular or missed periods ? '))
Concieve = (input('Difficulty in conceiving ?'))
Skin = (input('Acne or skin tags problem ?'))
Hairthin = (input('Hair thinning or hair loss problems ?'))
Patch = (input('Dark patches on the back of your neck or underarms ?'))
Tired = (input('Feeling of tiredness all the time ?'))
Mood = (input('Mood Swings more often than usual ?'))
Exercise = int(input('How many days in a week did you exercise ?'))
Eat = int(input('How many times a week did you eat outside ? '))
Can = (input('Did you consume canned food more often ?')) 	
City = (input('Did you move to a different city/location before experiencing the above conditions ?'))
PCOS = (input('You had/have PCOS ?'))
y_inputtest = [[age,Chin,Cheeks,Lips,Breast,Arms,Thigh,Exercise,Eat,PCOS,BMI,Weight,Period,Concieve,Skin,Hairthin,Patch,Tired,
                Mood,Can,City]]
for i in range(1,13):
    y_inputtest[0][-i]=g(y_inputtest[0][-i])
output = (loaded_model.predict(y_inputtest))
if(output == "True"):
  print("Predicted PCOS")
else :
  print("No PCOS")

Select your age at which you had PCOS22
Chin(Rate from 1-low to 5-high)2
Cheeks(Rate from 1-low to 5-high)2
Upper Lips(Rate from 1-low to 5-high)2
Between breast(Rate from 1-low to 5-high)2
Arms(Rate from 1-low to 5-high)2
Inner thighs(Rate from 1-low to 5-high)2
Were you overweight according to your BMI (Body Mass Index) ?NO
Have you experienced significant or rapid weight gain and faced difficulty in losing it ?NO
Have you experienced irregular or missed periods ? NO
Difficulty in conceiving ?NO
Acne or skin tags problem ?NO
Hair thinning or hair loss problems ?NO
Dark patches on the back of your neck or underarms ?NO
Feeling of tiredness all the time ?NO
Mood Swings more often than usual ?NO
How many days in a week did you exercise ?5
How many times a week did you eat outside ? 6
Did you consume canned food more often ?NO
Did you move to a different city/location before experiencing the above conditions ?NO
You had/have PCOS ?NO
No PCOS


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
